In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow import keras
from tensorflow.keras import layers as L
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [2]:
# Чтение DataFrame
df_models = pd.read_csv("данные для моделей.csv")
print("DataFrame 'данные для моделей.csv' успешно загружен.")

DataFrame 'данные для моделей.csv' успешно загружен.


In [3]:
print("--- Классификация: превышает ли значение IC50 медианное значение выборки ---")

# Определение целевой переменной для IC50
target_ic50_median_class = 'ic50_above_median' # Новая целевая колонка

# Вычисляем медиану для 'log_IC50, mM'
median_ic50 = df_models['log_IC50, mM'].median()
# Создаем бинарный столбец: 1, если log_CC50 > медианы, иначе 0
df_models[target_ic50_median_class] = (df_models['log_IC50, mM'] > median_ic50).astype(int)

print(f"Медианное значение log_IC50, mM: {median_ic50:.4f}")
print(f"Распределение классов для {target_ic50_median_class}:\n{df_models[target_ic50_median_class].value_counts()}")

# Признаки (X) и целевая переменная (y)
# Исключаем только исходные логарифмированные целевые переменные и только созданную бинарную целевую переменную для CC50.
X = df_models.drop(columns=['log_IC50, mM', 'log_CC50, mM', 'log_SI', target_ic50_median_class])
y = df_models[target_ic50_median_class]

# Разделение на обучающую и тестовую выборки
X_train_ic50_class, X_test_ic50_class, y_train_ic50_class, y_test_ic50_class = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y # stratify=y для сохранения пропорций классов
)

print(f"\nРазмер обучающей выборки (X_train_ic50_class): {X_train_ic50_class.shape}")
print(f"Размер тестовой выборки (X_test_ic50_class): {X_test_ic50_class.shape}")
print(f"Распределение классов в обучающей выборке:\n{y_train_ic50_class.value_counts(normalize=True)}")
print(f"Распределение классов в тестовой выборке:\n{y_test_ic50_class.value_counts(normalize=True)}")

--- Классификация: превышает ли значение IC50 медианное значение выборки ---
Медианное значение log_IC50, mM: 1.6542
Распределение классов для ic50_above_median:
ic50_above_median
0    484
1    483
Name: count, dtype: int64

Размер обучающей выборки (X_train_ic50_class): (773, 68)
Размер тестовой выборки (X_test_ic50_class): (194, 68)
Распределение классов в обучающей выборке:
ic50_above_median
0    0.500647
1    0.499353
Name: proportion, dtype: float64
Распределение классов в тестовой выборке:
ic50_above_median
0    0.5
1    0.5
Name: proportion, dtype: float64


In [4]:
# Метрики для GridSearchCV
scoring_metrics_class = {
    'Accuracy': 'accuracy',
    'F1': 'f1',
    'ROC_AUC': 'roc_auc'
}

In [5]:
# Логистическая Регрессия
print("\n===== Логистическая Регрессия для IC50 (классификация) =====")
pipeline_lr_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear')) # solver для избежания предупреждений
])

param_grid_lr_class = {
    'classifier__C': [0.1, 1, 10], # Параметр регуляризации
    'classifier__penalty': ['l1', 'l2'] # Тип регуляризации
}

grid_search_lr_class = GridSearchCV(
    pipeline_lr_class,
    param_grid_lr_class,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC', # Оптимизация по ROC_AUC
    n_jobs=-1,
    verbose=1
)

grid_search_lr_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для Логистической Регрессии:", grid_search_lr_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lr_class.best_score_)

# Оценка на тестовой выборке
y_pred_lr_class = grid_search_lr_class.predict(X_test_ic50_class)
y_proba_lr_class = grid_search_lr_class.predict_proba(X_test_ic50_class)[:, 1] # Вероятности для ROC AUC

accuracy_lr_class = accuracy_score(y_test_ic50_class, y_pred_lr_class)
f1_lr_class = f1_score(y_test_ic50_class, y_pred_lr_class)
roc_auc_lr_class = roc_auc_score(y_test_ic50_class, y_proba_lr_class)

print("\nМетрики на тестовой выборке (Логистическая Регрессия):")
print(f"Accuracy: {accuracy_lr_class:.4f}")
print(f"F1-score: {f1_lr_class:.4f}")
print(f"ROC AUC: {roc_auc_lr_class:.4f}")
print("\nClassification Report (Логистическая Регрессия):")
print(classification_report(y_test_ic50_class, y_pred_lr_class))


===== Логистическая Регрессия для IC50 (классификация) =====
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшие параметры для Логистической Регрессии: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC_AUC на кросс-валидации: 0.7242223144820547

Метрики на тестовой выборке (Логистическая Регрессия):
Accuracy: 0.6959
F1-score: 0.7122
ROC AUC: 0.7668

Classification Report (Логистическая Регрессия):
              precision    recall  f1-score   support

           0       0.72      0.64      0.68        97
           1       0.68      0.75      0.71        97

    accuracy                           0.70       194
   macro avg       0.70      0.70      0.69       194
weighted avg       0.70      0.70      0.69       194



In [6]:
# Random Forest Классификатор 
print("\n===== Random Forest Классификатор для IC50 (классификация) =====")
pipeline_rf_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid_rf_class = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_features': [0.6, 0.8, 1.0],
    'classifier__min_samples_leaf': [5, 10]
}

grid_search_rf_class = GridSearchCV(
    pipeline_rf_class,
    param_grid_rf_class,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_rf_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для Random Forest:", grid_search_rf_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_rf_class.best_score_)

# Оценка на тестовой выборке
y_pred_rf_class = grid_search_rf_class.predict(X_test_ic50_class)
y_proba_rf_class = grid_search_rf_class.predict_proba(X_test_ic50_class)[:, 1]

accuracy_rf_class = accuracy_score(y_test_ic50_class, y_pred_rf_class)
f1_rf_class = f1_score(y_test_ic50_class, y_pred_rf_class)
roc_auc_rf_class = roc_auc_score(y_test_ic50_class, y_proba_rf_class)

print("\nМетрики на тестовой выборке (Random Forest):")
print(f"Accuracy: {accuracy_rf_class:.4f}")
print(f"F1-score: {f1_rf_class:.4f}")
print(f"ROC AUC: {roc_auc_rf_class:.4f}")
print("\nClassification Report (Random Forest):")
print(classification_report(y_test_ic50_class, y_pred_rf_class))


===== Random Forest Классификатор для IC50 (классификация) =====
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Лучшие параметры для Random Forest: {'classifier__max_features': 0.6, 'classifier__min_samples_leaf': 10, 'classifier__n_estimators': 100}
Лучший ROC_AUC на кросс-валидации: 0.7837843974207611

Метрики на тестовой выборке (Random Forest):
Accuracy: 0.6856
F1-score: 0.6935
ROC AUC: 0.7881

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.70      0.66      0.68        97
           1       0.68      0.71      0.69        97

    accuracy                           0.69       194
   macro avg       0.69      0.69      0.69       194
weighted avg       0.69      0.69      0.69       194



In [7]:
# LightGBM Классификатор 
print("\n===== LightGBM Классификатор для IC50 (классификация) =====")
pipeline_lgbm_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(random_state=42, verbose=-1)) # verbose=-1 для отключения вывода логов
])

param_grid_lgbm_class = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__num_leaves': [20, 31],
    'classifier__reg_alpha': [0.1, 0.5],
    'classifier__reg_lambda': [0.1, 0.5]
}

grid_search_lgbm_class = GridSearchCV(
    pipeline_lgbm_class,
    param_grid_lgbm_class,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_lgbm_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для LightGBM:", grid_search_lgbm_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lgbm_class.best_score_)

# Оценка на тестовой выборке
y_pred_lgbm_class = grid_search_lgbm_class.predict(X_test_ic50_class)
y_proba_lgbm_class = grid_search_lgbm_class.predict_proba(X_test_ic50_class)[:, 1]

accuracy_lgbm_class = accuracy_score(y_test_ic50_class, y_pred_lgbm_class)
f1_lgbm_class = f1_score(y_test_ic50_class, y_pred_lgbm_class)
roc_auc_lgbm_class = roc_auc_score(y_test_ic50_class, y_proba_lgbm_class)

print("\nМетрики на тестовой выборке (LightGBM):")
print(f"Accuracy: {accuracy_lgbm_class:.4f}")
print(f"F1-score: {f1_lgbm_class:.4f}")
print(f"ROC AUC: {roc_auc_lgbm_class:.4f}")
print("\nClassification Report (LightGBM):")
print(classification_report(y_test_ic50_class, y_pred_lgbm_class))


===== LightGBM Классификатор для IC50 (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Лучшие параметры для LightGBM: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 200, 'classifier__num_leaves': 31, 'classifier__reg_alpha': 0.5, 'classifier__reg_lambda': 0.1}
Лучший ROC_AUC на кросс-валидации: 0.7824534772586721

Метрики на тестовой выборке (LightGBM):
Accuracy: 0.6753
F1-score: 0.6897
ROC AUC: 0.7664

Classification Report (LightGBM):
              precision    recall  f1-score   support

           0       0.69      0.63      0.66        97
           1       0.66      0.72      0.69        97

    accuracy                           0.68       194
   macro avg       0.68      0.68      0.67       194
weighted avg       0.68      0.68      0.67       194



In [8]:
# XGBoost Классификатор 
print("\n===== XGBoost Классификатор для IC50 (классификация) =====")
pipeline_xgb_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

param_grid_xgb_class = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__subsample': [0.7, 1.0],
    'classifier__colsample_bytree': [0.7, 1.0]
}

grid_search_xgb_class = GridSearchCV(
    pipeline_xgb_class,
    param_grid_xgb_class,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для XGBoost:", grid_search_xgb_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_xgb_class.best_score_)

# Оценка на тестовой выборке
y_pred_xgb_class = grid_search_xgb_class.predict(X_test_ic50_class)
y_proba_xgb_class = grid_search_xgb_class.predict_proba(X_test_ic50_class)[:, 1]

accuracy_xgb_class = accuracy_score(y_test_ic50_class, y_pred_xgb_class)
f1_xgb_class = f1_score(y_test_ic50_class, y_pred_xgb_class)
roc_auc_xgb_class = roc_auc_score(y_test_ic50_class, y_proba_xgb_class)

print("\nМетрики на тестовой выборке (XGBoost):")
print(f"Accuracy: {accuracy_xgb_class:.4f}")
print(f"F1-score: {f1_xgb_class:.4f}")
print(f"ROC AUC: {roc_auc_xgb_class:.4f}")
print("\nClassification Report (XGBoost):")
print(classification_report(y_test_ic50_class, y_pred_xgb_class))


===== XGBoost Классификатор для IC50 (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits


C:\Users\admin\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:04:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Лучшие параметры для XGBoost: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC_AUC на кросс-валидации: 0.7826591157759989

Метрики на тестовой выборке (XGBoost):
Accuracy: 0.6856
F1-score: 0.6995
ROC AUC: 0.7906

Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.70      0.64      0.67        97
           1       0.67      0.73      0.70        97

    accuracy                           0.69       194
   macro avg       0.69      0.69      0.68       194
weighted avg       0.69      0.69      0.68       194



In [9]:
# CatBoost Классификатор
print("\n===== CatBoost Классификатор для IC50 (классификация) =====")
pipeline_cat_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', CatBoostClassifier(random_state=42, verbose=0)) # verbose=0 отключает вывод логов CatBoost
])

param_grid_cat_class = {
    'classifier__iterations': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__depth': [4, 6],
    'classifier__l2_leaf_reg': [1, 3]
}

grid_search_cat_class = GridSearchCV(
    pipeline_cat_class,
    param_grid_cat_class,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_cat_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для CatBoost:", grid_search_cat_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_cat_class.best_score_)

# Оценка на тестовой выборке
y_pred_cat_class = grid_search_cat_class.predict(X_test_ic50_class)
y_proba_cat_class = grid_search_cat_class.predict_proba(X_test_ic50_class)[:, 1]

accuracy_cat_class = accuracy_score(y_test_ic50_class, y_pred_cat_class)
f1_cat_class = f1_score(y_test_ic50_class, y_pred_cat_class)
roc_auc_cat_class = roc_auc_score(y_test_ic50_class, y_proba_cat_class)

print("\nМетрики на тестовой выборке (CatBoost):")
print(f"Accuracy: {accuracy_cat_class:.4f}")
print(f"F1-score: {f1_cat_class:.4f}")
print(f"ROC AUC: {roc_auc_cat_class:.4f}")
print("\nClassification Report (CatBoost):")
print(classification_report(y_test_ic50_class, y_pred_cat_class))


===== CatBoost Классификатор для IC50 (классификация) =====
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Лучшие параметры для CatBoost: {'classifier__depth': 6, 'classifier__iterations': 200, 'classifier__l2_leaf_reg': 1, 'classifier__learning_rate': 0.01}
Лучший ROC_AUC на кросс-валидации: 0.7857746149953942

Метрики на тестовой выборке (CatBoost):
Accuracy: 0.7165
F1-score: 0.7264
ROC AUC: 0.8074

Classification Report (CatBoost):
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        97
           1       0.70      0.75      0.73        97

    accuracy                           0.72       194
   macro avg       0.72      0.72      0.72       194
weighted avg       0.72      0.72      0.72       194



In [10]:
#Простая Нейронная Сеть (Keras Sequential) 
print("\n===== Простая Нейронная Сеть для IC50 (классификация) =====")

# Функция для создания Keras-модели
def build_nn_model_class(meta, hidden_layers=1, neurons=32, activation='relu',
                         optimizer='adam', learning_rate=0.001):
    n_features = meta["n_features_in_"]
    
    model = keras.Sequential()
    model.add(L.Input(shape=(n_features,)))
    
    for _ in range(hidden_layers):
        model.add(L.Dense(neurons, activation=activation))
        
    model.add(L.Dense(1, activation='sigmoid')) # Выходной слой для бинарной классификации (1 нейрон, сигмоида)
    
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

pipeline_nn_class = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', KerasClassifier(
        model=build_nn_model_class,
        hidden_layers=1,
        neurons=32,
        activation='relu',
        optimizer='adam',
        learning_rate=0.001,
        batch_size=32,
        epochs=50,
        verbose=0,
        random_state=42,
        loss='binary_crossentropy' # Указываем loss здесь
    ))
])

param_grid_nn_class = {
    'classifier__hidden_layers': [1, 2],
    'classifier__neurons': [32, 64],
    'classifier__activation': ['relu'],
    'classifier__optimizer': ['adam'],
    'classifier__learning_rate': [0.001, 0.01],
    'classifier__batch_size': [32, 64],
    'classifier__epochs': [50, 100]
}

grid_search_nn_class = GridSearchCV(
    pipeline_nn_class,
    param_grid_nn_class,
    cv=3, # Кросс-валидация для NN - уменьшаем до 3
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_nn_class.fit(X_train_ic50_class, y_train_ic50_class)

print("Лучшие параметры для Нейронной Сети:", grid_search_nn_class.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_nn_class.best_score_)

# Оценка на тестовой выборке
y_pred_nn_class = grid_search_nn_class.predict(X_test_ic50_class)
y_proba_nn_class = grid_search_nn_class.predict_proba(X_test_ic50_class)[:, 1]

accuracy_nn_class = accuracy_score(y_test_ic50_class, y_pred_nn_class)
f1_nn_class = f1_score(y_test_ic50_class, y_pred_nn_class)
roc_auc_nn_class = roc_auc_score(y_test_ic50_class, y_proba_nn_class)

print("\nМетрики на тестовой выборке (Нейронная Сеть):")
print(f"Accuracy: {accuracy_nn_class:.4f}")
print(f"F1-score: {f1_nn_class:.4f}")
print(f"ROC AUC: {roc_auc_nn_class:.4f}")
print("\nClassification Report (Нейронная Сеть):")
print(classification_report(y_test_ic50_class, y_pred_nn_class))


===== Простая Нейронная Сеть для IC50 (классификация) =====
Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\admin\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Лучшие параметры для Нейронной Сети: {'classifier__activation': 'relu', 'classifier__batch_size': 32, 'classifier__epochs': 50, 'classifier__hidden_layers': 1, 'classifier__learning_rate': 0.001, 'classifier__neurons': 64, 'classifier__optimizer': 'adam'}
Лучший ROC_AUC на кросс-валидации: 0.7584471492348217

Метрики на тестовой выборке (Нейронная Сеть):
Accuracy: 0.7165
F1-score: 0.7236
ROC AUC: 0.7948

Classification Report (Нейронная Сеть):
              precision    recall  f1-score   support

           0       0.73      0.69      0.71        97
           1       0.71      0.74      0.72        97

    accuracy                           0.72       194
   macro avg       0.72      0.72      0.72       194
weighted avg       0.72      0.72      0.72       194



### Анализ результатов классификации: IC50 выше медианы
Цель классификации — определить, превышает ли значение IC50 медианное значение выборки. Медианное значение log_IC50 составляет 1.6542. Классы целевой переменной ic50_above_median хорошо сбалансированы: 484 случая для класса 0 и 483 для класса 1. Размеры обучающей и тестовой выборок также сбалансированы по классам.

1. Логистическая Регрессия
Лучшие параметры: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC AUC на кросс-валидации: 0.7242
Метрики на тестовой выборке:
Accuracy: 0.6959
F1-score: 0.7122
ROC AUC: 0.7668
Classification Report:
Precision (класс 0 / класс 1): 0.72 / 0.68
Recall (класс 0 / класс 1): 0.64 / 0.75
F1-score (класс 0 / класс 1): 0.68 / 0.71
Модель показала удовлетворительный ROC AUC на тестовой выборке (0.7668). Точность (Accuracy) составляет около 70%. Наблюдается небольшой дисбаланс в Precision и Recall: класс 1 имеет более высокий Recall (0.75) за счет несколько более низкой Precision (0.68) по сравнению с классом 0. Это означает, что модель хорошо выявляет случаи выше медианы, но может давать больше ложных срабатываний.

2. Random Forest Классификатор
Лучшие параметры: {'classifier__max_features': 0.6, 'classifier__min_samples_leaf': 10, 'classifier__n_estimators': 100}
Лучший ROC AUC на кросс-валидации: 0.7838
Метрики на тестовой выборке:
Accuracy: 0.6856
F1-score: 0.6935
ROC AUC: 0.7881
Classification Report:
Precision (класс 0 / класс 1): 0.70 / 0.68
Recall (класс 0 / класс 1): 0.66 / 0.71
F1-score (класс 0 / класс 1): 0.68 / 0.69
Random Forest продемонстрировал улучшенные показатели ROC AUC (0.7881) по сравнению с логистической регрессией. Accuracy и F1-score находятся в диапазоне 0.68-0.69, что сопоставимо с логистической регрессией. Precision и Recall для обоих классов сбалансированы.

3. LightGBM Классификатор
Лучшие параметры: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 200, 'classifier__num_leaves': 31, 'classifier__reg_alpha': 0.5, 'classifier__reg_lambda': 0.1}
Лучший ROC AUC на кросс-валидации: 0.7825
Метрики на тестовой выборке:
Accuracy: 0.6753
F1-score: 0.6897
ROC AUC: 0.7664
Classification Report:
Precision (класс 0 / класс 1): 0.69 / 0.66
Recall (класс 0 / класс 1): 0.63 / 0.72
F1-score (класс 0 / класс 1): 0.66 / 0.69
LightGBM показал ROC AUC на тестовой выборке (0.7664), сравнимый с логистической регрессией и несколько ниже, чем у Random Forest. Accuracy и F1-score также находятся в аналогичном диапазоне. Отмечается небольшое снижение Recall для класса 0 (0.63) и повышение для класса 1 (0.72).

4. XGBoost Классификатор
Лучшие параметры: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC AUC на кросс-валидации: 0.7827
Метрики на тестовой выборке:
Accuracy: 0.6856
F1-score: 0.6995
ROC AUC: 0.7906
Classification Report:
Precision (класс 0 / класс 1): 0.70 / 0.67
Recall (класс 0 / класс 1): 0.64 / 0.73
F1-score (класс 0 / класс 1): 0.67 / 0.70
XGBoost продемонстрировал один из лучших показателей ROC AUC на тестовой выборке (0.7906). Accuracy и F1-score сопоставимы с другими моделями на основе деревьев. Как и у LightGBM, наблюдается тенденция к более высокому Recall для класса 1 (0.73) и более низкому для класса 0 (0.64).

5. CatBoost Классификатор
Лучшие параметры: {'classifier__depth': 6, 'classifier__iterations': 200, 'classifier__l2_leaf_reg': 1, 'classifier__learning_rate': 0.01}
Лучший ROC AUC на кросс-валидации: 0.7858
Метрики на тестовой выборке:
Accuracy: 0.7165
F1-score: 0.7264
ROC AUC: 0.8074
Classification Report:
Precision (класс 0 / класс 1): 0.73 / 0.70
Recall (класс 0 / класс 1): 0.68 / 0.75
F1-score (класс 0 / класс 1): 0.71 / 0.73
CatBoost показал наивысшие результаты по всем ключевым метрикам на тестовой выборке. ROC AUC составил 0.8074, Accuracy — 0.7165, и F1-score — 0.7264. Precision и Recall для обоих классов хорошо сбалансированы и высоки, что указывает на эффективное распознавание обоих классов. CatBoost является лидером среди протестированных моделей для данной задачи классификации.

6. Простая Нейронная Сеть
Лучшие параметры: {'classifier__activation': 'relu', 'classifier__batch_size': 32, 'classifier__epochs': 50, 'classifier__hidden_layers': 1, 'classifier__learning_rate': 0.001, 'classifier__neurons': 64, 'classifier__optimizer': 'adam'}
Лучший ROC AUC на кросс-валидации: 0.7584
Метрики на тестовой выборке:
Accuracy: 0.7165
F1-score: 0.7236
ROC AUC: 0.7948
Classification Report:
Precision (класс 0 / класс 1): 0.73 / 0.71
Recall (класс 0 / класс 1): 0.69 / 0.74
F1-score (класс 0 / класс 1): 0.71 / 0.72
Нейронная сеть показала высокую Accuracy (0.7165) и F1-score (0.7236), сравнимые с CatBoost. ROC AUC на тестовой выборке (0.7948) также очень высок, хотя и немного ниже, чем у CatBoost. Метрики Precision и Recall хорошо сбалансированы для обоих классов. Модель продемонстрировала значительное улучшение ROC AUC на тестовой выборке по сравнению с кросс-валидацией, что указывает на хорошую обобщающую способность.

### Общий вывод по классификации: 
Все модели показывают достаточно хороший уровень производительности для классификации, с ROC AUC в диапазоне 0.76−0.81, что является удовлетворительным результатом. Accuracy и F1-score также находятся в приемлемых пределах (0.68−0.72).

Наилучшие результаты по всем ключевым метрикам (Accuracy, F1-score, ROC AUC) демонстрируют CatBoost и Простая Нейронная Сеть.

CatBoost является лидером по ROC AUC (0.8074), что указывает на его высокую способность к различению классов. Он также показывает наивысшие Accuracy (0.7165) и F1-score (0.7264) и хорошо сбалансированные Precision и Recall для обоих классов.
Нейронная Сеть демонстрирует очень конкурентоспособные метрики, сопоставимые с CatBoost по Accuracy (0.7165) и F1-score (0.7236), и высоким ROC AUC (0.7948). На основании анализа метрик, CatBoost и Простая Нейронная Сеть являются наиболее качественными решениями для этой задачи классификации.